In [2]:
import os
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import json

## Поставщики с диска

In [ ]:
path = 'файлы/'
res = np.array(os.listdir(path))
inn = np.unique(list(map(lambda x: re.findall('^(\d{3,})_\d{3,}', x)[0] if re.findall('^(\d{3,})_\d{3,}', x)!=[]  else '', res[res!='.DS_Store'])))[1:]


In [ ]:
nodes = []
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
for inn_num in inn:
    url = f'https://star-pro.ru/proverka-kontragenta/organization?searchQuery={inn_num}'
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, 'lxml')
    if soup.find('div', class_='ftfbn-note-bar__message')==None:
        name = re.sub('\s{2,}',' ', re.sub('[\n\r]|\s+',' ',soup.find('a', class_='ftfbn-link').text).strip())
    else: 
        url = f'https://star-pro.ru/proverka-kontragenta/organization?searchQuery=0{inn_num}'
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.text, 'lxml')
        name = re.sub('\s{2,}',' ', re.sub('[\n\r]|\s+',' ',soup.find('a', class_='ftfbn-link').text).strip())
        print(inn_num)
    nodes.append({'id': inn_num, 'name':name})

In [ ]:
with open('nodes.json', 'w') as f:
    f.write(json.dumps(nodes))

In [ ]:
%%file find_inf.py
import os
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import json


with open('records.json','r',encoding = 'utf-8') as fp: 
    method1 = json.load(fp)
with open('records2.json','r',encoding = 'utf-8') as fp: 
    method2 = json.load(fp)
method1.update(method2)


def find_inf(inf, d, method = method1):
    list_inf = []
    company = []
    for get_inf in inf:
        attr = {} #dict of links
        try: 
            attr['contract'] = re.findall(r'\d+', get_inf.find('div', class_="registry-entry__header-mid__number").text)[0]
        except AttributeError:
            attr['contract'] = ''
        try:
            attr['source'] = re.sub('\s{2,}',' ', re.sub('[\n\r]|\s+',' ',get_inf.find('div', class_="registry-entry__body-href").text)).strip()
            #re.sub('\s{2,}',' ', re.sub('[\n\r]|\s+',' ',get_inf.find('div', class_="registry-entry__body-href").text)).strip()
        except AttributeError:
            attr['source'] = ''
        company.append(attr['source'])
        try:
            attr['fz'] = re.findall(re.compile('\d+-\w+'), get_inf.find('div', class_="col-9 p-0 registry-entry__header-top__title text-truncate").text)[0]
        except AttributeError:
            attr['fz'] = ''
        try: 
            attr['price'] = re.sub('[\n\r]|\s','',get_inf.find("div", class_="price-block__value").text)
        except AttributeError:
            attr['price'] = ''
        attr['target'] = d
        try:
            attr['date'] = re.sub('[\n\r]|\s+',' ',get_inf.find("div", class_="data-block__value").text)
        except AttributeError:
            attr['date'] = ''
        try:
            attr['object'] = get_inf.find('div', class_="registry-entry__body").find("div", class_="registry-entry__body-value").text
        except AttributeError:
            attr['object'] = ''
        try:
            attr['method'] = method[attr['contract']]['class']
        except KeyError:
            attr['method'] = ''
        list_inf.append(attr)
    return list_inf, company 


def find_inn(num_inn):    
    url = f'https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&sortBy=UPDATE_DATE&pageNumber=1&sortDirection=false&recordsPerPage=_10&showLotsInfoHidden=false&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&participantName={num_inn}'
    nodes = {}
    links = []
    company_list = []
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, 'lxml')
    if response.status_code==404:
        return {num_inn: 'Not found'}
    else:
        print(url)
        pages = list(map(lambda x: int(x.text), soup.find_all('span', class_="link-text")))
        if pages == []:
            pages = 1
        else:
            pages = max(pages)
        for p in range(1, pages+1):
            if p==1:
                soup = soup
            else:
                url = f'https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&sortBy=UPDATE_DATE&pageNumber={p}&sortDirection=false&recordsPerPage=_10&showLotsInfoHidden=false&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&participantName={num_inn}'
                response = requests.get(url, headers = headers)
                soup = BeautifulSoup(response.text, 'lxml')
            inf = soup.find_all('div', class_="search-registry-entry-block box-shadow-search-input")
            link, company = find_inf(inf, num_inn)
            links.append(link)
            company_list.extend(company)
    return company_list, links

In [ ]:
from find_inf import find_inn
import multiprocessing

with multiprocessing.Pool(processes=8) as pool:
    res = []
    res = pool.map(find_inn, inn)

In [ ]:
%%file find_company.py
import os
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
from fuzzy_match import algorithims

def find_company(company):
    url = f'https://zakupki.gov.ru/epz/organization/search/results.html?searchString={company}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, 'lxml')
    d = {}
    if soup.find("div", class_="registry-entry__body")==None:
        d = {}
    else:
        prob_comp = soup.find_all("div", class_="registry-entry__header-mid__number")
        prob_comp = list(map(lambda x: re.sub('\s{2,}',' ', re.sub('[\n\r]|\s+',' ',x.text)).strip(), prob_comp))
        tr = list(map(lambda x: algorithims.trigram(x, company), prob_comp))
        if max(tr)>=0.8:
            index = np.argmax(tr)
            d = {}
            v = map(lambda x: x.text,soup.find_all("div", class_="registry-entry__body")[index].find_all("div", class_="registry-entry__body-value"))
            k = map(lambda x: x.text,soup.find_all("div", class_="registry-entry__body")[index].find_all("div", class_="registry-entry__body-title"))
            d = dict(zip(k,v))
            d['уровень'] = re.sub('\s{2,}',' ', re.sub('[\n\r]|\s+',' ',soup.find_all("div", class_="col d-flex flex-column registry-entry__right-block b-left")[index].find("div", class_="text-block__title").text).strip())
            d['название'] = prob_comp[index]
            d['tr'] = max(tr)
    return {company: d}
    
    

In [ ]:
from find_company import find_company
import multiprocessing
c = list(set(companies))
with multiprocessing.Pool(processes=8) as pool:
    res = []
    res = pool.map(find_company, c)

In [ ]:
import json
with open('company2.json', 'w') as f:
    f.write(json.dumps(res))

In [ ]:
with open('company2.json', 'r') as fp:
    all_company = json.load(fp)
res_list = {}
for i in all_company:
    res_list.update(i)

In [ ]:
nodes_dict = {}
for company, inf in res_list.items():
    if inf!={}:
        nodes_dict.update({inf['ИНН']:company})
nodes_list = []
for inf, company in nodes_dict.items():
    nodes_list.append({'id': inf, 'name': company})
with open('nodes.json', 'r') as f:
    nodes_deliver = json.load(f)
nodes_list.extend(nodes_deliver)
n = {x['id']: x['name'] for x in nodes_list}
nodes_list_without_rep = [{'id': k, 'name': v} for k, v in n.items()]


In [ ]:
nodes = {}
for link in links:
    name = link['source']
    try:
        link['source'] = res_list[name]['ИНН']
        link['address'] = res_list[name]['Местонахождение']
        link['level'] = res_list[name]['уровень']
    except KeyError:
        link['source'] = name
with_inn = []
for link in links:
    if re.findall(r'\d{5,}',link['source']) != []:
        with_inn.append(link)


In [ ]:
for link in with_inn:
    pat1 = re.compile('\d+,\d+₽')
    if re.findall(pat1,str(link['price'])) == [] and link['price']!='':
        link['price'] = re.sub(',', '.',link['price'])
        print(link)
        try:
            link['price'] = float(re.findall(r'\d+.\d+',str(link['price']))[0])*exchange_rate.loc[link['date'][3:],'курс']
        except KeyError:
            link['price'] = float(re.findall(r'\d+.\d+',str(link['price']))[0])*exchange_rate.loc[link["date"][4:],'курс']
        print(link)
    elif link['price']!='':
        link['price'] = re.sub(',', '.',link['price'])
        link['price'] = float(re.findall(r'\d+.\d+',str(link['price']))[0])

In [ ]:
import json
with open('nodes_links3.json', 'w') as f:
    f.write(json.dumps({'nodes': nodes_list_without_rep,
                       'links': with_inn_rubls}))

In [3]:
with open('nodes_links3.json', 'r') as f:
    g = json.load(f)


In [4]:
g

{'nodes': [{'id': '7813604570',
   'name': 'ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ НЕТИПОВОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "АКАДЕМИЯ ТАЛАНТОВ" САНКТ-ПЕТЕРБУРГА'},
  {'id': '2301088612',
   'name': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЛЕЧЕБНО-ОЗДОРОВИТЕЛЬНЫЙ КОМПЛЕКС "ВИТЯЗЬ"'},
  {'id': '3435881134',
   'name': 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "СРЕДНЯЯ ШКОЛА № 36 ИМЕНИ ГЕРОЯ СОВЕТСКОГО СОЮЗА В.Г. МИЛОВАТСКОГО Г. ВОЛЖСКОГО ВОЛГОГРАДСКОЙ ОБЛАСТИ"'},
  {'id': '5504004684',
   'name': 'бюджетное учреждение здравоохранения Омской области "Клинический кожно-венерологический диспансер"'},
  {'id': '8609000629',
   'name': 'МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "РАДУЖНЫЙТЕПЛОСЕТЬ" ГОРОДСКОГО ОКРУГА РАДУЖНЫЙ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ'},
  {'id': '3906220339',
   'name': 'ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ КАЛИНИНГРАДСКОЙ ОБЛАСТИ "ЭКОЛОГИЧЕСКИЙ ЦЕНТР "ЕКАТ-КАЛИНИНГРАД"'},
  {'id': '7838407446',
   'name': 'Общество с ограниченной ответственностью "Жилкомсервис № 2 Адмиралтейс